O fluxo do programa é  

1) baixar 1433 arquivos TAR públicos com os dados e as imagens das patentes publicadas entre 2001 e 2021 diretamente do repositório de patentes dos Estados Unidos conhecido como Bulk Data Storage System (BDSS) no diretório `imgs/raw` criado por nós em nosso Drive do Google e  

2) descompactar esses arquivos TAR no mesmo diretório `imgs/raw`. Ao descompactar esses arquivos temos uma estrutura de diretórios no formato `IYYYYMMDD` e em cada diretório vários subdiretórios no formato `UTIL99999` (entre outros). Em cada subdiretório desses encontram-se vários arquivos no formato `USXX999999-YYYYMMDD.ZIP` que somados chegam a cerca de 4 milhões de arquivos.  

3) Descompactamos esses arquivos ZIP para o diretório `imgs/lake`. Ao descompactá-los temos um diretório para cada patente no formato `USXX999999-YYYYMMDD`. Em cada diretório, encontram-se um arquivo `.XML` com as informações textuais da patente e imagens das patentes no formato `TIFF` que podem variar de 1 a algumas centenas de imagens. (TODO: calcular a média de imagens por patente).  

4) Lemos o arquivo `.XML` e obtemos a classe IPC da patente. Então, convertemos as imagens presentes de TIFF para JPEG (ou seria PNG melhor?) e as enviamos para o diretório correspondente à classe IPC da patente, i.e. `imgs/ipc/X99`. No total temos 131 diferentes diretórios. Um para cada classe IPC. (Deveríamos aprofundar mais e usar as 647 subclasses IPC? O que isso traria de vantagem?)  

Agora temos as imagens convertidas e divididas em suas respectivas classes, prontas para serem manipuladas pelo algoritmo de extração de características (features extraction).  

5) Onde e como guardar essas features? Em um banco de dados? Como?  
.  
.  
**A estrutura de diretórios / arquivos é mais ou menos a seguinte:**  

---
`imgs/raw/`  
`---- I20210112.tar (file)(untar here)`  

`---- I20210112 (dir)`  
`-------- UTIL10891 (dir)`  
`------------ US108911000-20210112.ZIP (unzip to imgs/lake)`  
`------------ US108911001-20210112.ZIP`  
`------------ US108911002-20210112.ZIP`  
`------------ etc`  
`-------- UTIL10892 (dir)`  
`-------- UTIL10893 (dir)`  
`-------- etc.`  

---
`imgs/lake`  
`----US108911000-20210112 (dir)`  
`--------US108911000-20210112.XML (file)`  
`--------US108911000-20210112-D00000.TIF (file)`  
`--------US108911000-20210112-D00001.TIF (file)`  
`--------US108911000-20210112-D00002.TIF (file)`  
`--------etc.`  
Read XML, get IPC class(es) convert from TIFF to JPEG and send it to IPC folder.  

---
`imgs/ipc`  
`---- A01`  
`-------- US108911000-20210112-D00000.JPG`  
`-------- US108911000-20210112-D00001.JPG`  
`-------- etc.`  
`---- A02`  
`---- A03`    
`etc.`  

---

Mais informações encontram-se ao longo do código.  

**Passos realizados até agora:**
1. <strike>criar diretórios para as classes IPC</strike>
2. <strike>listar e salvar links com arquivos TAR</strike>
3. <strike>baixar arquivos TAR</strike>
4. <strike>listar e descompactar arquivos TAR</strike>
5. <strike>listar subarquivos ZIP</strike>
6. <strike>descompactar subarquivos ZIP para um diretório comum</strike>
7. <strike>apagar tudo que não for mais necessário para economizar espaço.</strike> Ainda mantemos os arquivos TAR no Drive. Estamos apagando apenas os subarquivos ZIP por enquanto.

Os seguintes passos encontram-se em outro notebook:
8. ler full text, determinar classe IPC da patente, transformar e enviar imagem para o respectivo diretório específico

---
---
---
###0. setup

Conectamo-nos ao drive, onde salvamos os dados provenientes do Bulk Data Storage System (BDSS).

In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalamos o wget para poder baixar os dados.  
TODO: testar um outro módulo que já esteja instalado no colab para fazer o download dos dados.

In [ ]:
#install uninstalled modules
!pip install wget

Importamos as bibliotecas e módulos necessários para o funcionamento do código.

In [2]:
#import libraries and modules
import os
import pickle
#TODO: test other module than wget
#import wget

from bs4 import BeautifulSoup
import re
import requests

import tarfile
from zipfile import ZipFile

Declaramos algumas constantes que apontam para alguns diretórios em nosso disco.  

**`DATA_DIR`** é onde salvamos listas de paths, features, etc. em formato pickle.  

**`RAW_IM`** é onde salvamos e descompactamos os arquivos TAR provenientes diretamente do Bulk Data Storage System (BDSS).

**`IPC_IM`** é o diretório onde criamos a estrutura de subdiretórios baseados nas classes IPC. Neles salvamos as imagens convertidas de TIFF para JPG (ou seria melhor PNG?). Os algoritmos de extração de features lêem esses subdiretórios e buscam aqui as imagens JPG (/PNG).

**`IM_LAKE`** é onde descompactamos os subarquivos ZIP que contêm diretórios para cada patente com os arquivos XML e as imagens em formato TIFF.

In [3]:
#important paths

DATA_DIR = '/content/drive/My Drive/octimine/data/' 
IM_LAKE = '/content/drive/My Drive/octimine/imgs/lake/' 
IPC_IM = '/content/drive/My Drive/octimine/imgs/ipc/'
RAW_IM = '/content/drive/My Drive/octimine/imgs/raw/'

---
---
---
###1. create IPC classes folders

Lista com todas as 131 classes IPC.  
Será que deveríamos usar as 647 subclasses IPC? Isso traria algum benefício na hora de extrair as features?  
Mais informações: https://www.wipo.int/

In [ ]:
#create folders related to IPC classes
#https://ipcpub.wipo.int/

ipc_classes = [
  'A01','A21','A22','A23','A24','A41','A42','A43','A44','A45','A46','A47','A61','A62','A63','A99',
  'B01','B02','B03','B04','B05','B06','B07','B08','B09','B21','B22','B23','B24','B25','B26','B27','B28','B29','B30','B31','B32','B33','B41','B42','B43','B44','B60','B61','B62','B63','B64','B65','B66','B67','B68','B81','B82','B99',
  'C01','C02','C03','C04','C05','C06','C07','C08','C09','C10','C11','C12','C13','C14','C21','C22','C23','C25','C30','C40','C99',
  'D01','D02','D03','D04','D05','D06','D07','D21','D99',
  'E01','E02','E03','E04','E05','E06','E21','E99',
  'F01','F02','F03','F04','F15','F16','F17','F21','F22','F23','F24','F25','F26','F27','F28','F41','F42','F99',
  'G01','G02','G03','G04','G05','G06','G07','G08','G09','G10','G11','G12','G16','G21','G99',
  'H01','H02','H03','H04','H05','H99'
]

Cria os diretórios baseando-se na lista acima, utilizando-se a biblioteca `os`.  

Lê a lista.
Para cada ítem da lista, cria-se um path IPC_IM + ítem.
Verifica se já existe o diretório.  
Caso afirmativo, volta-se para o início do loop.  
Caso negativo, criar-se o diretório usando o path.

In [ ]:
#mkdir
for ipc_class in ipc_classes:
  ipc_path = os.path.join(IPC_IM, ipc_class)
  ipc_exists = os.path.isdir(ipc_path)

  if ipc_exists:
    print(f'Folder {ipc_class} exists already! Moving on...')
  else:
    os.mkdir(ipc_path)
    print(f'Folder {ipc_path} successfully created.')

---
---
---
###2. list paths of TAR files to be downloaded

Lista todos os anos a serem pesquisados.  
Cria-se uma lista vazia que armazenará os endereços dos arquivos a serem baixados.

In [4]:
#list TAR files paths
#https://www.crummy.com/software/BeautifulSoup/bs4/doc.ptbr/ 

tar_files = []

#from 2001 to 2010 the files are ZIP
#from 2011 onwards the files are TAR

years = [
  '2021',
  '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011',
  '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001'
]

Popula a lista tar_files.  

Lê a lista years e determina-se a url a ser pesquisada.  
Utilizando-se a biblioteca BeautifulSoup analisa-se (parse) o documento .html e extraem-se os links que contêm a palavra-chave (key) que é o ano em questão.  
Caso algo seja encontrado, é adicionado à lista tar_files.
Imprime-se o número de ítens na lista.

In [5]:
for year in years:
  url = f'https://bulkdata.uspto.gov/data/patent/grant/redbook/{year}/'
  soup = BeautifulSoup(requests.get(url).text, 'html.parser')

  #https://www.crummy.com/software/BeautifulSoup/bs4/doc.ptbr/#os-argumentos-palavras-chave
  #keyword, common to all files to be downloaded
  keyword = year

  for x in soup.find_all(href=re.compile(keyword)):
      tar_files.append(url + x.get('href'))

print(len(tar_files))

1433


Salva-se a lista `tar_files` para uso posterior caso se dê prosseguimento com a operação apenas mais tarde.

In [6]:
#save TAR files list for later use
pickle.dump(
    tar_files,
    open(DATA_DIR + 'TAR-files.pickle', 'wb'))

---
---
---
###3. download TAR files

In [ ]:
#open TAR files list
tar_files = pickle.load(open(DATA_DIR + 'TAR-files.pickle', 'rb'))

In [ ]:
#download TAR files
for tar_file in tar_files:
  file_name = tar_file[-12:]
  file_path = RAW_IM + file_name
  file_exists = os.path.isfile(file_path)

  if file_exists:
    print(f'File {file_name} has been downloaded already! Moving on...')
  else:
    wget.download(tar_file, file_path)
    print(f'Downloading file {file_name}. Please wait...')    
    print(f'File {file_name} successfully downloaded.')

---
---
---
###4. list, untar and delete downloaded TAR source files

In [ ]:
#list tar files
untar_files = []

for root, dirs, files in os.walk(RAW_IM):
  for x in files:
    #check only for .tar files
    if x.endswith('.tar'):
      untar_files.append(os.path.join(root, x))
  #break to list only first level folder
  #https://stackoverflow.com/a/20868760/3499881
  break

len(untar_files)

108

Salva-se a lista `untar_files` para uso posterior caso se dê prosseguimento com a operação apenas mais tarde.

In [ ]:
#save unTAR files list for later use
pickle.dump(
    untar_files,
    open(DATA_DIR + 'unTAR-files.pickle', 'wb'))

---
####4.2 untar and delete

In [ ]:
#open unTAR files list
untar_files = pickle.load(open(DATA_DIR + 'unTAR-files.pickle', 'rb'))

In [ ]:
#untar files
for untar_file in untar_files:
  folder_name = 'I' + untar_file[-12:-4]
  folder_path = RAW_IM + folder_name
  folder_exists = os.path.isdir(folder_path)

  if folder_exists:
    print(f'Folder {folder_name} exists already! Moving on...')

  else:
    print(f'Untaring file {untar_file}. Please wait...')    
    untar = tarfile.open(untar_file)
    untar.extractall(RAW_IM)
    untar.close()
    print(f'File {untar_file} successfully untared.')

    #delete original tar files to save space
    #print(f'Deleting {untar_file}. Please wait...')
    #os.remove(untar_file)
    #print(f'File {untar_file} successfully deleted.')

---
---
---
###5. list, unzip and delete ZIP (sub)files


In [7]:
#list ZIP subfiles
zip_files = []

for root, dirs, files in os.walk(RAW_IM):
  for x in files:
    if x.endswith('.ZIP'):
      zip_files.append(os.path.join(root, x))

len(zip_files)

391056

In [8]:
#save ZIP files list for later use
pickle.dump(
    zip_files,
    open(DATA_DIR + 'ZIP-files.pickle', 'wb'))

---
####5.2 unzip and delete

In [ ]:
#open ZIP files list
zip_files = pickle.load(open(DATA_DIR + 'ZIP-files.pickle', 'rb'))

In [ ]:
#unzip ZIP subfiles
#TODO: verificar se tarfile descompactaria arquivos zip e vice-versa
for zip_file in zip_files:
  folder_name = zip_file[-23:-4]
  supp_name = zip_file[-28:-4]
  folder_path = IM_LAKE + folder_name
  folder_supp = IM_LAKE + supp_name
  folder_exists = os.path.isdir(folder_path)
  supp_exists = os.path.isdir(folder_supp)

  if folder_exists or supp_exists:
    print(f'Folder {folder_name} exists already! Moving on...')
    #delete original zip subfile
    print(f'Deleting {zip_file}. Please wait...')
    os.remove(zip_file)
    print(f'File {zip_file} successfully deleted.')

  else:
    print(f'1. Unzipping file {zip_file}. Please wait...')
    ZipFile(zip_file, 'r').extractall(path=IM_LAKE)
    print(f'2. File {zip_file} successfully unzipped.')
    #delete original zip subfile
    print(f'3. Deleting {zip_file}. Please wait...')
    os.remove(zip_file)
    print(f'4. File {zip_file} successfully deleted.')